## WAVE data retrieval
### thanks to Copernicus

In [1]:
pip install copernicusmarine

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import modules
import copernicusmarine

# Define parameters to run the extraction
dataset_id = "cmems_mod_glo_wav_my_0.2_PT3H-i"  # Ensure the dataset ID is correct

# Load dataframe
request_dataframe = copernicusmarine.read_dataframe(
    dataset_id=dataset_id,
    minimum_longitude=-42.56,  # Corrected for 42.68°W
    maximum_longitude=-42.56,  # Assuming a single point; expand as needed
    minimum_latitude=59.46,    # Corrected for 59.48°N
    maximum_latitude=59.46,    # Assuming a single point; expand as needed
    # Update the variable names based on the dataset documentation
    variables=[
        "sea_surface_primary_swell_wave_mean_period",  # Example variable name for sea_surface_wind_wave_significant_height
        "sea_surface_primary_swell_wave_significant_height",  # Example variable name for sea_surface_wind_wave_mean_period
        # Add additional variables as needed for primary/secondary swells
    ],
    start_datetime="2015-01-01T00:00:00",  # Set the correct start date
    end_datetime="2019-12-31T23:59:59",    # Set the correct end date
    # Include depth if the dataset supports it and is relevant for your analysis
    # minimum_depth=0,
    # maximum_depth=6
)

request_dataframe


INFO - 2024-03-14T10:22:14Z - Dataset version was not specified, the latest one was selected: "202112"
INFO - 2024-03-14T10:22:14Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-03-14T10:22:17Z - Service was not specified, the default one was selected: "arco-time-series"
username:Abort


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Reset the index to make all current index columns into regular columns
df_reset = request_dataframe.reset_index()

# Assuming the datetime column is named 'time' after resetting the index. If it's named differently, adjust accordingly.
# Set this column as the new index
df_reset.set_index('time', inplace=True)

# Now, ensure the index is in datetime format
df_reset.index = pd.to_datetime(df_reset.index)

# Plotting
plt.figure(figsize=(10, 6))

# Plot VTM01_SW1
plt.plot(df_reset.index, df_reset['VTM01_SW1'], label='VTM01_SW1', color='blue')

# Plot VHM0_SW1 on the same graph
plt.plot(df_reset.index, df_reset['VHM0_SW1'], label='VHM0_SW1', color='red')

# Adding some plot decorations
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series of VTM01_SW1 and VHM0_SW1')
plt.legend()
plt.grid(True)

# Adjusting x-axis date format for better readability
plt.gcf().autofmt_xdate()

plt.show()


AttributeError: 'NoneType' object has no attribute 'reset_index'

In [ ]:
request_dataframe.head()

In [ ]:
import pandas as pd

df_reset = request_dataframe.reset_index()

# Drop the 'latitude' and 'longitude' columns
df_reset = df_reset.drop(['latitude', 'longitude'], axis=1)

# Now set 'time' as the new index
df_reset = df_reset.set_index('time')

# Now we reindex the DataFrame to have an hourly frequency, inserting NaNs for the missing times
df_hourly = df_reset.resample('H').asfreq()

# Interpolate the NaNs using cubic spline interpolation for each column you're interested in
df_hourly['VTM01_SW1'] = df_hourly['VTM01_SW1'].interpolate(method='spline', order=3)
df_hourly['VHM0_SW1'] = df_hourly['VHM0_SW1'].interpolate(method='spline', order=3)

# Show the result
print(df_hourly)

In [ ]:
plt.figure(figsize=(10, 6))

# Plot VTM01_SW1
plt.plot(df_hourly.index, df_hourly['VTM01_SW1'], label='VTM01_SW1', color='blue')

# Plot VHM0_SW1 on the same graph
plt.plot(df_hourly.index, df_hourly['VHM0_SW1'], label='VHM0_SW1', color='red')

# Adding some plot decorations
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series of VTM01_SW1 and VHM0_SW1')
plt.legend()
plt.grid(True)

# Adjusting x-axis date format for better readability
plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
import numpy as np

# Perform the calculation and create a new DataFrame with the resulting series (in W/m)

# Constants for the calculation
rho_water = 1000  # Density of water in kg/m^3
g = 9.81  # Acceleration due to gravity in m/s^2

J_wave = (rho_water * g**2) / (64 * np.pi) * df_hourly['VTM01_SW1'] * df_hourly['VHM0_SW1']**2

J_wave = J_wave/1e9 #to convert in GW/m

result_df = pd.DataFrame(J_wave, columns=['Wave_Energy_Density'])

print(result_df)

# Save the result to a CSV file
result_df.to_csv('../../data/wave_series_goff.csv', header=False, index=False)

In [ ]:
import pandas as pd

# Example input data
length_of_installation = 60000.0  # meters, example value
unit_rated_power_gw = 0.0059  # GW per unit
unit_width = 260.0  # meters
space_between = 10.0  # meters

# Calculate total space required per unit (width + space)
total_space_per_unit = unit_width + space_between

# Calculate the number of units that can be installed
number_of_units = length_of_installation // total_space_per_unit

# Convert the list of wave energy density values into a DataFrame for easier processing
wave_energy_df = result_df

# Calculate the total energy potential for each wave energy density value
wave_energy_df['TotalEnergyPotentialGW'] = wave_energy_df['Wave_Energy_Density'] * length_of_installation

# Calculate the total rated energy capacity based on the number of units
total_rated_capacity_gw = number_of_units * unit_rated_power_gw

# Check when the total energy potential exceeds the total rated capacity
wave_energy_df['ExceedsRatedPower'] = wave_energy_df['TotalEnergyPotentialGW'] > total_rated_capacity_gw

print(wave_energy_df)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming wave_energy_df is already defined with the following columns:
# 'WaveEnergyDensityGWperM', 'TotalEnergyPotentialGW', 'ExceedsRatedPower'

# For visualization, calculating the total rated capacity again for the plot
total_rated_capacity_gw = number_of_units * unit_rated_power_gw

# Plotting
plt.figure(figsize=(10, 6))

# Plot Total Energy Potential GW
plt.bar(wave_energy_df.index, wave_energy_df['TotalEnergyPotentialGW'], color='blue', label='Total Energy Potential (GW)')

# Plot a line for the Total Rated Capacity GW Threshold
plt.axhline(y=total_rated_capacity_gw, color='g', linestyle='-', label='Rated Power * Number of Units (GW)')

# Optionally, highlight bars where the energy potential exceeds the rated capacity
for index, row in wave_energy_df.iterrows():
    if row['ExceedsRatedPower']:
        plt.bar(index, row['TotalEnergyPotentialGW'], color='red')

plt.xlabel('Index')
plt.ylabel('Energy (GW)')
plt.title('Wave Energy Potential vs. Rated Capacity Threshold')
plt.legend()
plt.grid(True)

plt.show()
